In [1]:
!pip install datasets -U
!pip install transformers -U
!pip install torchaudio -U
!pip install librosa -U
!pip install jiwer -U
!apt install git-lfs -U
!pip install hazm -U
!pip install pydub -U
!pip install pythainlp -U
!pip install huggingface-hub -U
!pip install num2fawords -U
!pip install parsivar -U
!pip install -q kaggle

     |████████████████████████████████| 346 kB 8.5 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 140 kB 79.4 MB/s 
     |████████████████████████████████| 127 kB 77.0 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 271 kB 59.2 MB/s 
     |████████████████████████████████| 144 kB 53.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.2 MB 8.3 MB/s 


In [2]:
import os
import re
import pandas as pd
import torch
import os
import random
import numpy as np
import string
import json
from IPython.display import display
import librosa
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers.trainer_utils import get_last_checkpoint
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2Config, TrainingArguments, Trainer
from parsivar import Normalizer
import num2fawords
import re
import string
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass
from tqdm import tqdm
import glob
import hazm

from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

In [3]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files


files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexahab9","key":"5c6bb822c9fb2e9db96b78c8f310ff39"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!mkdir -p /content/data/shemo
!kaggle datasets download mansourehk/shemo-persian-speech-emotion-detection-database
!unzip shemo-persian-speech-emotion-detection-database.zip -d /content/data/shemo/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/shemo/transcript/final script/F24A22.tra  
  inflating: /content/data/shemo/transcript/final script/F24A23.tra  
  inflating: /content/data/shemo/transcript/final script/F24A24.tra  
  inflating: /content/data/shemo/transcript/final script/F24A25.tra  
  inflating: /content/data/shemo/transcript/final script/F24A26.tra  
  inflating: /content/data/shemo/transcript/final script/F24A27.tra  
  inflating: /content/data/shemo/transcript/final script/F24A28.tra  
  inflating: /content/data/shemo/transcript/final script/F24A29.tra  
  inflating: /content/data/shemo/transcript/final script/F24A30.tra  
  inflating: /content/data/shemo/transcript/final script/F24A31.tra  
  inflating: /content/data/shemo/transcript/final script/F24A32.tra  
  inflating: /content/data/shemo/transcript/final script/F24A33.tra  
  inflating: /content/data/shemo/transcript/final script/F24A34.tra  
  inflating: /content/data/shemo/transc

In [11]:
abs_path_to_data = os.path.join("/content", "data", "shemo")
!ls {abs_path_to_data}

female	male  transcript


In [12]:
data = []
for txtfile in tqdm(glob.glob(f"{abs_path_to_data}/transcript/final text/*.ort")):
    with open(txtfile, "r", encoding="utf-8") as f:
        text = f.read()
        _id = txtfile.split("/")[-1].split(".")[0]
        male_path = f"{abs_path_to_data}/male/{_id}.wav"
        female_path = f"{abs_path_to_data}/female/{_id}.wav"
        
        if "M" in _id and os.path.exists(male_path):
            data.append({
                "_id": _id,
                "sentence": text,
                "path": male_path.strip(),
                "gender": "M"
            })
        
        if "F" in _id and os.path.exists(female_path):
            data.append({
                "_id": _id,
                "sentence": text,
                "path": female_path.strip(),
                "gender": "F"
            })
        
#     break
    
df = pd.DataFrame(data)
df.head()

100%|██████████| 3000/3000 [00:00<00:00, 37541.99it/s]


,_id,sentence,path,gender
0,F02W03,ماشین ریسندگی؟\n,/content/data/shemo/female/F02W03.wav,F
1,F01H06,مشتریاش روزبه‌روز بیشتر می‌شه.\n,/content/data/shemo/female/F01H06.wav,F
2,F29N07,شما هیچ‎وقت در مراسم مذهبی سنت آنسلم شرکت نکرد...,/content/data/shemo/female/F29N07.wav,F
3,F10A42,به نظرم خبر نداری توی این شهر چه خبره.\n,/content/data/shemo/female/F10A42.wav,F
4,M28A38,فرمانروا عاشق دوناسله\n,/content/data/shemo/male/M28A38.wav,M


In [13]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?", 
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
#     "ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",
        
    # "ها": "  ها", "ئ": "ی",
    "۱۴ام": "۱۴ ام",
        
    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None
    
    return text + " "

In [14]:
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df["prev_sentence"] = df["sentence"]
df["sentence"] = df["sentence"].apply(lambda t: normalizer(t))
df = df.dropna(subset=["sentence"])
print(f"Step 2: {len(df)}")

# df = df[["prev_sentence", "sentence", "path"]]
# df = df[["sentence", "path"]]
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


Step 0: 2838
Step 1: 2838
Step 2: 2838


,_id,sentence,path,gender
0,M25W07,باز چطور ممکنه,/content/data/shemo/male/M25W07.wav,M
1,F31A10,این ادا اصولها چیه دیگه,/content/data/shemo/female/F31A10.wav,F
2,M12N16,ولی تو این فاصله اتفاقات مهمی افتاده,/content/data/shemo/male/M12N16.wav,M
3,M25A68,دیگه چه نقشه ای داری,/content/data/shemo/male/M25A68.wav,M
4,M25A65,شکستت بهت قدرت اخلاقی نمی ده,/content/data/shemo/male/M25A65.wav,M


In [15]:
main_vocab = ["ح", "چ", "ج", "ث", "ت", "پ", "ب", "آ", "ا", "ش", "س", "ژ", "ز", "ر", "ذ", "د", "خ", "ق", "ف", "غ", "ع", "ظ", "ط", "ض", "ص", "ی", "ه", "و", "ن", "م", "ل", "گ", "ک"]
text = " ".join(df["sentence"].values.tolist())
vocab = list(sorted(set(text)))

for v in main_vocab:
    if v not in vocab:
        print("v", v)

print(len(main_vocab), len(vocab))
print(vocab)

33 36
[' ', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [16]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["sentence"], "\n")
speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

2372
البته قربان البته اینجاست خوب نگاش کنین  



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  from ipykernel import kernelapp as app


In [17]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=101, stratify=df["gender"])

train_df = train_df[["path", "sentence"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "sentence"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)

(2554, 2)
(284, 2)


In [18]:
save_path = "/".join(abs_path_to_data.split('/'))
print(save_path)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(test_df.shape)

/content/data/shemo
(2554, 2)
(284, 2)


In [19]:
common_voice_train = load_dataset("csv", data_files={"train": "/content/data/shemo/train.csv"}, delimiter="\t")["train"]
common_voice_test = load_dataset("csv", data_files={"test": "/content/data/shemo/test.csv"}, delimiter="\t")["test"]

Using custom data configuration default-b9d43daa73986f25


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b9d43daa73986f25/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e53b770f9c9881d5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e53b770f9c9881d5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
print(common_voice_train)
print(common_voice_test)

Dataset({
    features: ['path', 'sentence'],
    num_rows: 2554
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 284
})


In [21]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), 'Can\'t pick more elements than there are in the dataset.'

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)

        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(df.head())

In [22]:
show_random_elements(common_voice_train.remove_columns(['path']), num_examples=10)

,sentence
0,گفتم خفه شو غلط زیادی نکن دختر
1,تو چی می خوای بگی مستر همشو به یه خواب تبدیل کنی
2,من می خوام همین ماه برم پاریس بعد هم برمی گردم...
3,امیدوارم داشته باشید چون دیشب من خواب شما را د...
4,الان حقش نیست که با ورود یکی دیگه به اون کم مح...


In [23]:
print(common_voice_train[0]['sentence'])
print(common_voice_test[0]['sentence'])

آره خلاصه دماغش چاق چاقه 
اون وقت الان سر کی این وسط بی کلاه موند من 


In [24]:
def normalizer(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["sentence"].lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None
    
    batch["sentence"] = text
    
    return batch

In [25]:
common_voice_train = common_voice_train.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})
common_voice_test = common_voice_test.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})

  0%|          | 0/2554 [00:00<?, ?ex/s]

  0%|          | 0/284 [00:00<?, ?ex/s]

In [26]:
def extract_all_chars(batch):
    all_text = ' '.join(batch['sentence'])
    vocab = list(set(all_text))
    return {'vocab': [vocab], 'all_text': [all_text]}

In [27]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
vocab_list = list(sorted(set(vocab_train['vocab'][0]) | set(vocab_test['vocab'][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [' ', '\u0307']]
print(len(vocab_list))
print(vocab_list)

35
['ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [29]:
special_vocab = ['<pad>', '<s>', '</s>', '<unk>', '|']
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

40
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'ء': 5, 'آ': 6, 'ئ': 7, 'ا': 8, 'ب': 9, 'ت': 10, 'ث': 11, 'ج': 12, 'ح': 13, 'خ': 14, 'د': 15, 'ذ': 16, 'ر': 17, 'ز': 18, 'س': 19, 'ش': 20, 'ص': 21, 'ض': 22, 'ط': 23, 'ظ': 24, 'ع': 25, 'غ': 26, 'ف': 27, 'ق': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'پ': 34, 'چ': 35, 'ژ': 36, 'ک': 37, 'گ': 38, 'ی': 39}


In [30]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [31]:
tokenizer = Wav2Vec2CTCTokenizer('vocab.json', bos_token='<s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', word_delimiter_token='|',
                                 do_lower_case=False)

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [32]:
text = 'از مهمونداری کنار بکشم'
print(' '.join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

ا ز | م ه م و ن د ا ر ی | ک ن ا ر | ب ک ش م
از مهمونداری کنار بکشم


In [33]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))

40


In [34]:
target_sampling_rate = 16_000

min_duration_in_seconds = 5.0
max_duration_in_seconds = 10.0

In [35]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch['path'])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)

    batch['speech'] = speech_array
    batch['sampling_rate'] = target_sampling_rate
    batch['duration_in_seconds'] = len(batch['speech']) / target_sampling_rate
    batch['target_text'] = batch['sentence']
    return batch

def filter_by_max_duration(batch):
    return min_duration_in_seconds <= batch['duration_in_seconds'] <= max_duration_in_seconds

# check that all files have the correct sampling rate
def prepare_dataset(batch):
    assert (
        len(set(batch['sampling_rate'])) == 1), f'Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}.'

    batch['input_values'] = processor(batch['speech'], sampling_rate=batch['sampling_rate'][0]).input_values

    with processor.as_target_processor():
        batch['labels'] = processor(batch['target_text']).input_ids

    return batch

In [36]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

  0%|          | 0/2554 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=22050, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.


  0%|          | 0/284 [00:00<?, ?ex/s]

In [37]:
print(common_voice_train[0]['sampling_rate'])
print(common_voice_test[0]['sampling_rate'])

16000
16000


In [38]:
print(f'Split sizes [BEFORE]: {len(common_voice_train)} train and {len(common_voice_test)} validation.')

Split sizes [BEFORE]: 2554 train and 284 validation.


In [39]:
_common_voice_train = common_voice_train.filter(filter_by_max_duration)
_common_voice_test = common_voice_test.filter(filter_by_max_duration)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
print(f'Split sizes [AFTER]: {len(_common_voice_train)} train and {len(_common_voice_test)} validation.')

Split sizes [AFTER]: 521 train and 63 validation.


In [41]:
_common_voice_train = _common_voice_train.map(prepare_dataset, remove_columns=_common_voice_train.column_names, batch_size=8, batched=True)
_common_voice_test = _common_voice_test.map(prepare_dataset, remove_columns=_common_voice_test.column_names, batch_size=8, batched=True)

  0%|          | 0/66 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/8 [00:00<?, ?ba/s]

In [38]:
##_common_voice_train.set_format(type='torch', columns=['input_values', 'labels'])
##_common_voice_test.set_format(type='torch', columns=['input_values', 'labels'])

In [42]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{'input_values': feature['input_values']} for feature in features]
        label_features = [{'input_ids': feature['labels']} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors='pt')

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(label_features, padding=self.padding, max_length=self.max_length_labels, pad_to_multiple_of=self.pad_to_multiple_of_labels, return_tensors='pt')

        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch['labels'] = labels
        return batch

In [43]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [44]:
def compute_metrics(pred):
    
    pred_logits = pred.predictions
    print('pred_logits', pred_logits)
    pred_ids = np.argmax(pred_logits, axis=-1)
    print('pred_ids',pred_ids)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    print('pred_str',pred_str)

    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print('label_str',label_str)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str[index]}')
        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str}')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {'wer': wer}

In [45]:
wer_metric = load_metric('wer')

In [46]:
print(len(processor.tokenizer.get_vocab()))

40


In [47]:
model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-large-xlsr-53', attention_dropout=0.1, hidden_dropout=0.1, feat_proj_dropout=0.0,
                                       mask_time_prob=0.05, layerdrop=0.1, gradient_checkpointing=True, ctc_loss_reduction='mean', 
                                       ctc_zero_infinity=True, bos_token_id=processor.tokenizer.bos_token_id, eos_token_id=processor.tokenizer.eos_token_id,
                                       pad_token_id=processor.tokenizer.pad_token_id, vocab_size=len(processor.tokenizer.get_vocab()))

model.freeze_feature_extractor()

model.to(torch.device('cuda'))

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [48]:
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/hr3', group_by_length=True, per_device_train_batch_size=10, per_device_eval_batch_size=10,
                                  save_strategy='no', eval_steps=10, gradient_accumulation_steps=2, evaluation_strategy='steps', num_train_epochs=2,
                                  learning_rate=1e-3, no_cuda=False, fp16=True, logging_steps=10, warmup_steps=1, save_total_limit=1,)
                                  #load_best_model_at_end=True), save_steps=10,

trainer = Trainer(model=model, data_collator=data_collator, args=training_args, compute_metrics=compute_metrics, train_dataset=_common_voice_train,
                  eval_dataset=_common_voice_test, tokenizer=processor.feature_extractor)

Using amp half precision backend


In [49]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 521
  Num Epochs = 2
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 2
  Total optimization steps = 52


Step,Training Loss,Validation Loss,Wer
10,10.437000,3.741825,1.000000
20,3.397900,3.014696,1.000000
30,3.115400,2.975119,1.000000
40,2.988700,2.967918,1.000000
50,2.945100,2.967168,1.000000


***** Running Evaluation *****
  Num examples = 63
  Batch size = 10


pred_logits [[[   3.855     -0.4753    -0.4504  ...   -0.4077    -0.4485     0.11475]
  [   3.867     -0.4673    -0.4517  ...   -0.4014    -0.439      0.1227 ]
  [   3.87      -0.469     -0.438   ...   -0.4167    -0.4229     0.1343 ]
  ...
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]]

 [[   3.87      -0.4976    -0.4526  ...   -0.4072    -0.4722     0.1465 ]
  [   3.877     -0.4712    -0.4504  ...   -0.4104    -0.438      0.1302 ]
  [   3.887     -0.472     -0.4482  ...   -0.4175    -0.431      0.12476]
  ...
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]
  [-100.      -100.      -100.      ... -100.      -100.      -100.     ]]

 [[   3.854     -0.4834    -0.4326  ...   -0.4292    -0.4585     0.1436 ]
  [   3.86

***** Running Evaluation *****
  Num examples = 63
  Batch size = 10


...

***** Running Evaluation *****
  Num examples = 63
  Batch size = 10


pred_logits [[[ 5.0469e+00 -3.2812e+00 -3.2676e+00 ...  2.1423e-02 -1.0342e+00
    1.0645e+00]
  [ 5.0430e+00 -3.2852e+00 -3.2676e+00 ...  2.2415e-02 -1.0312e+00
    1.0693e+00]
  [ 5.0391e+00 -3.2734e+00 -3.2676e+00 ...  2.1469e-02 -1.0244e+00
    1.0674e+00]
  ...
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]]

 [[ 5.0391e+00 -3.2891e+00 -3.2676e+00 ...  2.7573e-02 -1.0391e+00
    1.0723e+00]
  [ 5.0391e+00 -3.2832e+00 -3.2656e+00 ...  2.6749e-02 -1.0342e+00
    1.0693e+00]
  [ 5.0352e+00 -3.2891e+00 -3.2676e+00 ...  2.4323e-02 -1.0332e+00
    1.0674e+00]
  ...
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.000

***** Running Evaluation *****
  Num examples = 63
  Batch size = 10


pred_logits [[[ 4.9219e+00 -3.4043e+00 -3.3809e+00 ... -1.0425e-01 -8.8672e-01
    9.9658e-01]
  [ 4.9219e+00 -3.4062e+00 -3.3809e+00 ... -1.0388e-01 -8.8477e-01
    1.0010e+00]
  [ 4.9141e+00 -3.3965e+00 -3.3789e+00 ... -1.0327e-01 -8.7695e-01
    9.9902e-01]
  ...
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]]

 [[ 4.9102e+00 -3.4102e+00 -3.3809e+00 ... -9.6802e-02 -8.9062e-01
    1.0059e+00]
  [ 4.9141e+00 -3.4043e+00 -3.3789e+00 ... -9.8694e-02 -8.8574e-01
    1.0029e+00]
  [ 4.9102e+00 -3.4082e+00 -3.3789e+00 ... -1.0052e-01 -8.8428e-01
    9.9951e-01]
  ...
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.0000e+02
   -1.0000e+02]
  [-1.0000e+02 -1.0000e+02 -1.0000e+02 ... -1.0000e+02 -1.000



Training completed. Do not forget to share your model on huggingface.co/models =)




In [47]:
metrics = train_result.metrics
max_train_samples = len(_common_voice_train)
metrics['train_samples'] = min(max_train_samples, len(_common_voice_train))

trainer.save_model()

trainer.log_metrics('train', metrics)
trainer.save_metrics('train', metrics)
trainer.save_state()

Saving model checkpoint to /content/drive/MyDrive/hr3
Configuration saved in /content/drive/MyDrive/hr3/config.json
Model weights saved in /content/drive/MyDrive/hr3/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/hr3/preprocessor_config.json


***** train metrics *****
  epoch                    =         2.0
  total_flos               = 210384833GF
  train_loss               =      4.4056
  train_runtime            =  0:02:07.31
  train_samples            =         535
  train_samples_per_second =       8.404
  train_steps_per_second   =       0.424


In [51]:
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/hr45', group_by_length=True, per_device_train_batch_size=10, per_device_eval_batch_size=10,
                                  save_strategy='no', eval_steps=10, evaluation_strategy='steps', num_train_epochs=2,
                                  learning_rate=1e-3, no_cuda=False, fp16=True, logging_steps=10, save_total_limit=1,)
                                  #load_best_model_at_end=True), save_steps=10, gradient_accumulation_steps=2, warmup_steps=1,

trainer = Trainer(model=model, data_collator=data_collator, args=training_args, compute_metrics=compute_metrics, train_dataset=_common_voice_train,
                  eval_dataset=_common_voice_test, tokenizer=processor.feature_extractor)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [52]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 535
  Num Epochs = 2
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 108


Step,Training Loss,Validation Loss,Wer
10,9.519500,3.281933,1.000000
20,3.251100,2.994526,1.000000
30,2.940900,3.015971,1.000000
40,2.959900,2.961955,1.000000
50,2.948300,2.952144,1.000000
60,2.952800,2.934839,1.000000
70,2.967500,2.944411,1.000000
80,2.978200,2.959808,1.000000
90,2.914500,2.939537,1.000000
100,2.994700,2.927743,1.000000


***** Running Evaluation *****
  Num examples = 49
  Batch size = 10


pred_logits [[[ 4.430e+00 -8.608e-01 -5.869e-01 ... -7.393e-01 -7.207e-01 -6.427e-02]
  [ 4.438e+00 -8.633e-01 -6.035e-01 ... -7.383e-01 -7.153e-01 -5.243e-02]
  [ 4.438e+00 -8.667e-01 -6.064e-01 ... -7.422e-01 -7.168e-01 -4.208e-02]
  ...
  [ 4.430e+00 -8.486e-01 -6.089e-01 ... -7.407e-01 -7.290e-01 -5.347e-02]
  [ 4.430e+00 -8.501e-01 -6.089e-01 ... -7.397e-01 -7.295e-01 -5.383e-02]
  [ 4.430e+00 -8.481e-01 -6.094e-01 ... -7.402e-01 -7.295e-01 -5.426e-02]]

 [[ 4.430e+00 -8.481e-01 -6.021e-01 ... -7.593e-01 -7.100e-01 -5.545e-02]
  [ 4.434e+00 -8.525e-01 -6.035e-01 ... -7.520e-01 -7.090e-01 -5.661e-02]
  [ 4.438e+00 -8.438e-01 -6.021e-01 ... -7.554e-01 -7.061e-01 -5.280e-02]
  ...
  [ 4.430e+00 -8.506e-01 -6.089e-01 ... -7.480e-01 -7.236e-01 -5.429e-02]
  [ 4.430e+00 -8.506e-01 -6.089e-01 ... -7.480e-01 -7.236e-01 -5.429e-02]
  [ 4.430e+00 -8.506e-01 -6.089e-01 ... -7.480e-01 -7.236e-01 -5.429e-02]]

 [[ 4.426e+00 -8.628e-01 -5.903e-01 ... -7.422e-01 -7.114e-01 -6.549e-02]
  [ 4.438e

***** Running Evaluation *****
  Num examples = 49
  Batch size = 10


...

***** Running Evaluation *****
  Num examples = 49
  Batch size = 10


pred_logits [[[   4.895    -4.375    -4.293  ...    0.1743   -0.7715    1.137 ]
  [   4.902    -4.383    -4.305  ...    0.174    -0.7725    1.152 ]
  [   4.9      -4.383    -4.305  ...    0.1765   -0.7725    1.153 ]
  ...
  [   4.895    -4.375    -4.3    ...    0.1732   -0.7764    1.149 ]
  [   4.895    -4.375    -4.3    ...    0.1731   -0.7764    1.149 ]
  [   4.895    -4.375    -4.3    ...    0.1731   -0.7764    1.148 ]]

 [[   4.9      -4.375    -4.297  ...    0.1727   -0.7705    1.146 ]
  [   4.902    -4.38     -4.297  ...    0.1735   -0.771     1.148 ]
  [   4.9      -4.375    -4.297  ...    0.1729   -0.769     1.147 ]
  ...
  [   4.89     -4.375    -4.297  ...    0.1744   -0.773     1.1455]
  [   4.89     -4.375    -4.297  ...    0.1744   -0.773     1.1455]
  [   4.89     -4.375    -4.297  ...    0.1744   -0.773     1.1455]]

 [[   4.895    -4.375    -4.293  ...    0.1765   -0.769     1.142 ]
  [   4.895    -4.38     -4.297  ...    0.175    -0.7686    1.148 ]
  [   4.9      -4.38

***** Running Evaluation *****
  Num examples = 49
  Batch size = 10


pred_logits [[[   5.01     -4.43     -4.332  ...    0.1475   -0.95      1.287 ]
  [   5.02     -4.438    -4.344  ...    0.1473   -0.951     1.303 ]
  [   5.016    -4.438    -4.344  ...    0.1498   -0.951     1.304 ]
  ...
  [   5.01     -4.43     -4.344  ...    0.1462   -0.955     1.3   ]
  [   5.01     -4.43     -4.344  ...    0.1462   -0.955     1.3   ]
  [   5.01     -4.43     -4.344  ...    0.1461   -0.955     1.299 ]]

 [[   5.016    -4.43     -4.336  ...    0.1458   -0.9487    1.296 ]
  [   5.02     -4.434    -4.34   ...    0.1469   -0.9497    1.299 ]
  [   5.016    -4.43     -4.34   ...    0.1461   -0.9478    1.298 ]
  ...
  [   5.01     -4.43     -4.34   ...    0.1473   -0.9517    1.296 ]
  [   5.01     -4.43     -4.34   ...    0.1473   -0.9517    1.296 ]
  [   5.01     -4.43     -4.34   ...    0.1473   -0.9517    1.296 ]]

 [[   5.01     -4.43     -4.332  ...    0.1497   -0.9473    1.29  ]
  [   5.01     -4.434    -4.34   ...    0.1487   -0.947     1.298 ]
  [   5.016    -4.43



Training completed. Do not forget to share your model on huggingface.co/models =)


